plot_change.py: Generate significance of electrode weight changes at neuroanatomical sites

## Pseudocode:
    . Import
    . Input symptom, mode, perdur
    . Define functions
    . Load Parameters
    . For Loop through seizures

## Import

In [1]:
import csv
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import butter,filtfilt
from scipy.stats import ttest_1samp

#
import pandas as pd
# from plot_change_module import *

In [2]:
neuroanat_list = ['frontalpole', #FRONTAL LOBE
    'parstriangularis',
    'parsopercularis',
    'parsorbitalis',
    'rostralmiddlefrontal',
    'caudalmiddlefrontal',
    'lateralorbitofrontal',
    'superiorfrontal',
    'medialorbitofrontal',
    'precentral',
    'postcentral', # PARIETAL LOBE
    'inferiorparietal',   
    'superiorparietal',
    'supramarginal',
    'temporalpole', # TEMPORAL LOBE
    'middletemporal',
    'superiortemporal',
    'inferiortemporal',
    'parahippocampal',               
    'Right-Hippocampus',
    'Left-Hippocampus',
    'Right-Amygdala',
    'Left-Amygdala',
    'entorhinal',
    'bankssts',
    'fusiform', # OCCIPITAL LOBE                
    'lingual'];
    # 'Right-Inf-Lat-Vent', # OTHER
    # 'Right-Cerebral-White-Matter',
    # 'Left-Cerebral-White-Matter',
    # 'Right-choroid-plexus',
    # 'Right-Putamen',
    # 'Right-VentralDC'];
    
abv_neuroanat_list = ['front-pole', #FRONTAL LOBE
    'parstri',
    'parsop',
    'parsorb',
    'rost-midfront',
    'caud-midfront',
    'latorb-front',
    'sup-front',
    'medorb-front',
    'precentral',
    'postcentral', # PARIETAL LOBE
    'inf-par',   
    'sup-par',
    'supra-marg',
    'temp-pole', # TEMPORAL LOBE
    'mid-temp',
    'sup-temp',
    'inf-temp',
    'parahip',               
    'R-Hip',
    'L-Hip',
    'R-Amyg',
    'L-Amyg',
    'entorhinal',
    'bankssts',
    'fusiform', # OCCIPITAL LOBE                
    'lingual'];
    # 'Right-Inf-Lat-Vent', # OTHER
    # 'Right-Cerebral-White-Matter',
    # 'Left-Cerebral-White-Matter',
    # 'Right-choroid-plexus',
    # 'Right-Putamen',
    # 'Right-VentralDC'];


## Input symptom, mode, and duration

In [3]:
sx_input = 'lhx'
mx_input = '2'
perdur_input = '8'

## Define Functions

In [4]:
def sem_w8s(LL,at_onset,before_onset,after_onset):
    row_num = np.shape(LL)[0]
    LL_meandiff = np.empty(row_num,)
    # print(int(at_onset))
    # print(LL[0,int(at_onset)-1])
    # print(LL[:,int(at_onset):int(after_onset)])
    # LL_meandiff[:] = np.mean(LL[:,int(at_onset):int(after_onset)],0) - np.mean(LL[:,int(before_onset):int(at_onset)],0)
    # print(LL_meandiff)
    for row in range(0,row_num-1): #for each channel in LL   
         LL_meandiff[row] = np.mean(LL[row,int(at_onset):int(after_onset)]) - np.mean(LL[row,int(before_onset):int(at_onset)])
    #     # print(LL_meandiff[row])
    #     print(np.mean(LL[row,int(at_onset):int(after_onset)]))
    return LL_meandiff

In [5]:
def ll_transform(llw,fs,d,bl_start,bl_stop):    
    if bl_start == 0:
        sample_bl_start = 0
    elif bl_start > 0:
        sample_bl_start = (fs*bl_start)[0] #0 because it's python not matlab
    sample_bl_end = (fs*bl_stop)[0]-1
    
    L = int(np.round(llw * fs) - 1) # number of samples to calculate line length
    
    col_len = np.shape(d)[1]-L
    
    LL = np.empty([np.shape(d)[0],col_len])
    
    LL[:] = np.NaN        
    
    for col_1 in range(0,col_len):
        LL[:,col_1] = np.sum(np.abs(np.diff(d[:,col_1:col_1+L])),1)


    for row_1 in range(0,np.shape(d)[0]):    
        LL_nanmean = np.nanmean(LL[row_1,sample_bl_start:sample_bl_end])
        LL_nanstd = np.nanstd(LL[row_1,sample_bl_start:sample_bl_end])
                                              
        LL[row_1,:] = (LL[row_1,:] - LL_nanmean)/LL_nanstd
                    
    return LL


In [6]:
def open_xl(xl_name): 
    
    df = pd.read_excel(xl_name, index_col=0, engine='openpyxl')

    return df


In [7]:
def filt(d,fs):
    d_t = d.transpose()
    
    butter_array = np.array([1,(round(fs[0]/2)-1)])
    b,a = butter(2,butter_array/(fs[0]/2),btype='bandpass',output='ba')
    
    filt_d = filtfilt(b,a,d_t,axis=0,padtype='odd',padlen=3*(max(len(b),len(a))-1)).transpose()

    return filt_d


In [8]:
def load_elecs_anat(pt_path):
                
    #load anatomy and electrode matrix
    os.chdir(pt_path + 'Imaging/elecs')
    e_mat = loadmat('clinical_elecs_all.mat')
    anat_col = e_mat['anatomy'][:,3]
            
    return anat_col


In [9]:
def get_params(df_params,pt):
    params_bl_start = df_params.loc[pt]['BLstart']
    params_bl_stop = df_params.loc[pt]['BLstop']
    params_llw = df_params.loc[pt]['llw']

    return params_bl_start, params_bl_stop, params_llw

In [10]:
def get_ptsz(avg_change_path): #Load list of patient and seizure names
    os.chdir(avg_change_path)
    
    df_params = open_xl('OPSCEAparams.xlsx') # Ndimensions and params_list

    with open('sz_name_list.csv','r') as sz_names:
        sz_names_list = list(csv.reader(sz_names))
        sz_rows = np.size(sz_names_list,0) #find row num 
        pt_data = []
        sz_data= []
        for r in range(0,sz_rows):  #collect symptom data for specfic seizure in list
            pt_data.append(sz_names_list[r][0]) 
            sz_data.append(sz_names_list[r][1]) 
    return pt_data, sz_data, df_params

## For loop through patients in list matching seizure

In [ ]:
avg_change_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/OPSCEADATA/avg_change_folders/'

pt_data,sz_data,df_params = get_ptsz(avg_change_path) #Ehsan


#plot prep
# subplot_idx = 1
xtick_labels = []
# plt.figure(figsize = (40, 20)) 


# plt.plot()


# for neuro_struc in neuroanat_list:

#For loop through patients
for i in range(len(pt_data)):  #Ehsan
    pt_i = pt_data[i]          #Ehsan
    sz_name = sz_data[i]       #Ehsan

    pt_path = avg_change_path + pt_i + '/'
    os.chdir(pt_path)
    pt_dir = os.listdir(pt_path)

    for sz_name in sz_data: 

        if sz_name in pt_dir: #patients in list matching seizure
            
#             ax = plt.subplot(len(sz_data)+1,1,subplot_idx)
#             plt.ylabel(sz_name)

#             subplot_idx += 1
            
            print(sz_name)
                
            # PARAMS
            blstart, blstop, llw = get_params(df_params,pt_i) # 2 = params_llw

            # ANAT 
            anat = load_elecs_anat(pt_path)
            
            
            # SEMIOLOGY
            sz_path = pt_path + sz_data[i]
            os.chdir(sz_path)

            sx_vec = pd.read_csv(sz_name + '_mat.csv',usecols = [sx_input]).values.tolist() #load semiology matrix .csv             #WANT TO JUST LOAD 1 COLUMN SX_INPUT            
            
            if np.float64(mx_input) in sx_vec:
            
                sz_mat = loadmat(sz_name +'.mat') #load frame speed and ppEEG
                fs = sz_mat['fs'].flatten()
                d = sz_mat['ppEEG'][0:np.shape(anat)[0],:]
                
                badch_mat = loadmat(sz_name + '_badch.mat')
                badch = badch_mat['bad_chs']
                
                badch = np.delete(badch,range(np.shape(anat)[0],np.shape(badch)[0]),0)

                for a_i in range(0,np.shape(anat)[0]):
                    if np.size(anat[a_i]) > 0: 
                        if anat[a_i][0] not in neuroanat_list:
                            badch[a_i][0] = 1 #prepare for anatomy not in neuroanat list to be deleted in d and anat
                
                bad_logical = np.any(badch,1)
                d = d[~bad_logical,:]
                anat = anat[~bad_logical]

                filt_d = filt(d,fs) #Filter out < 1 Hz (and up to nyquist)
    
                LL = ll_transform(llw,fs,filt_d,blstart,blstop) 
               
                at_onset = np.round(((sx_vec.index(np.float64(mx_input)))/5) * fs)
                before_onset = np.round(((sx_vec.index(np.float64(mx_input))/5) - float(perdur_input)) * fs)
                after_onset = np.round(((sx_vec.index(np.float64(mx_input))/5) + float(perdur_input)) * fs)

                
                LL_meandiff = sem_w8s(LL,at_onset,before_onset,after_onset)
                
                pval = np.empty(np.shape(neuroanat_list))
                pval[:] = np.NaN

                
                
                #string comparison 
                anat_index = [np.NaN] * np.shape(anat)[0]

                
                anat_list = np.empty(np.shape(anat)).tolist()
                anat_w8s = []
                for a in range(0,np.shape(anat)[0]):
                    if len(anat[a]) > 0:
                        anat_list[a] = anat[a][0]
                                            

                for n_l in range(0,len(neuroanat_list)):

#                         #clump together all the ones that match same neuroanat
                    for a_i in range(0,len(anat_list)):
                        if neuroanat_list[n_l] == anat_list[a_i][:]:
                            anat_w8s.append(LL_meandiff[a_i])
                            # if neuroanat_list[n_l] not in xtick_labels:
                            #      xtick_labels.append(neuroanat_list[n_l]) 
                    tstat, pval[n_l] = ttest_1samp(anat_w8s,0)
                    # plt.plot(n_l,0)
                    # plt.axvline(x=n_l)
                    print(neuroanat_list)
                    if pval[n_l] < .001:
                        print('     SIGNIFICANT PVAL: ',pval[n_l])
                        # plt.plot(n_l,w8,marker="x",color="red") #red x if significant 
                    elif pval[n_l] < .08 and pval[n_l] >= .001:
                        # plt.plot(n_l,w8,marker="x",color="yellow") # gray circle if trending towards significance
                        print('     TRENDING TOWARDS SIGNIFICANT PVAL',pval[n_l])

                    
                    
                    
#                     for w8 in anat_w8s:
#                         # plt.plot(n_l,w8,marker="o",color="gray") # gray circle if trending towards significance
#                         if pval[n_l] < .001:
#                             print('     SIGNIFICANT PVAL: ',pval[n_l])
#                             # plt.plot(n_l,w8,marker="x",color="red") #red x if significant 
#                         elif pval[n_l] < .08 and pval[n_l] >= .001:
#                             # plt.plot(n_l,w8,marker="x",color="yellow") # gray circle if trending towards significance
#                             print('     TRENDING TOWARDS SIGNIFICANT PVAL',pval[n_l])


                    
                    
                    
#                     if pval[n_l] < .001:
#                         for w8 in anat_w8s:
#                             plt.plot(n_l,w8,marker="x",color="red") #red x if significant 
#                     elif pval[n_l] < .08 and pval[n_l] >= .001:
#                         for w8 in anat_w8s:
#                             plt.plot(n_l,w8,marker="x",color="yellow") # gray circle if trending towards significance
#                     elif pval[n_l] >= .08 and pval[n_l] < .8:
#                         for w8 in anat_w8s:
#                             plt.plot(n_l,w8,marker="o",color="gray") # gray circle if trending towards significance

                    anat_w8s = []

# plt.xticks(range(0,len(neuroanat_list)),abv_neuroanat_list,fontsize=8)
# plt.tick_params(axis='x',[0:len(neuroanat_list)],neuroanat_list,fontsize=8)

#                         print(neuroanat_list[n_l],': ')
#                         if pval[n_l] < .001:
#                             print('     SIGNIFICANT PVAL: ',pval[n_l])
#                         elif pval[n_l] < .08:
#                             print('     TRENDING TOWARDS SIGNIFICANT PVAL',pval[n_l])

#                     if len(anat_w8s) > 0:
#                         print(neuroanat_list[n_l],': ')
#                         print('      w8s: ',anat_w8s)
#                         print('      pval: ',pval[n_l])     
#                         print(' ')
                # plt.subplot(subplot_idx)
                
# plt.setp(ax.set_xticklabels(xtick_labels))
                                            
#                         anat_w8s = LL_meandiff[anat_index]
#                         
#                         plot() 
        #subplot outside the for loop 
        #nl = x val, anat_w8s
         

EC91_03


/Users/nataliasucher/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/nataliasucher/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/nataliasucher/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/nataliasucher/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/nataliasucher/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


['frontalpole', 'parstriangularis', 'parsopercularis', 'parsorbitalis', 'rostralmiddlefrontal', 'caudalmiddlefrontal', 'lateralorbitofrontal', 'superiorfrontal', 'medialorbitofrontal', 'precentral', 'postcentral', 'inferiorparietal', 'superiorparietal', 'supramarginal', 'temporalpole', 'middletemporal', 'superiortemporal', 'inferiortemporal', 'parahippocampal', 'Right-Hippocampus', 'Left-Hippocampus', 'Right-Amygdala', 'Left-Amygdala', 'entorhinal', 'bankssts', 'fusiform', 'lingual']
['frontalpole', 'parstriangularis', 'parsopercularis', 'parsorbitalis', 'rostralmiddlefrontal', 'caudalmiddlefrontal', 'lateralorbitofrontal', 'superiorfrontal', 'medialorbitofrontal', 'precentral', 'postcentral', 'inferiorparietal', 'superiorparietal', 'supramarginal', 'temporalpole', 'middletemporal', 'superiortemporal', 'inferiortemporal', 'parahippocampal', 'Right-Hippocampus', 'Left-Hippocampus', 'Right-Amygdala', 'Left-Amygdala', 'entorhinal', 'bankssts', 'fusiform', 'lingual']
     TRENDING TOWARDS 